# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

In [2]:
from azureml.train.automl import AutoMLConfig

In [3]:
from azureml.core import Model

In [4]:
from azureml.automl.core.shared import constants
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import Webservice, AciWebservice

In [5]:
import joblib

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.
```
In this prooject I am using heart failure clinical records dataset from UCI machine learning repository which follow ups death events based on 12 clinical features.
```
TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

`UCI ML Repository for this dataset is` - [link](https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv)

In [6]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-azure-capstone-project'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [7]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()
print(ws)

Workspace name: quick-starts-ws-143143
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-143143
Workspace.create(name='quick-starts-ws-143143', subscription_id='9a7511b8-150f-4a58-8528-3e7d50216c31', resource_group='aml-quickstarts-143143')


In [8]:
compute_cluster_name = "Saturn-AI"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
except ComputeTargetException:
    compute_cluster_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_cluster_config)

compute_cluster.wait_for_completion(show_output=True)

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [9]:
DATASET_URI = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'

In [10]:
key = 'Heart Failure Clinical Records'

if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("Dataset exists in the datastore.")

else:
    dataset = Dataset.Tabular.from_delimited_files(DATASET_URI)
    dataset = dataset.register(ws,key)

df = dataset.to_pandas_dataframe()
print(f'Dataframe shape:', df.shape)

train, test = train_test_split(df, test_size=.33, shuffle=True)
print(f'Train count: ', len(train))
print(f'Test count: ', len(test))
train.to_csv('train.csv',index = False)
test.to_csv('test.csv',index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])
datastore.upload_files(files = ['./test.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore,'train.csv')])
test = Dataset.Tabular.from_delimited_files([(datastore,'test.csv')])

Dataframe shape: (299, 13)
Train count:  200
Test count:  99
Uploading an estimated of 1 files
Uploading ./train.csv
Uploaded ./train.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Uploading an estimated of 1 files
Uploading ./test.csv
Uploaded ./test.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [11]:
# TODO: Put your automl settings here
automl_settings = {} ## Defined in altogether with config

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=compute_cluster,
    task="classification",
    training_data=train,
    label_column_name="DEATH_EVENT",
    primary_metric="accuracy",
    n_cross_validations=5,
    experiment_timeout_minutes=30,
    enable_early_stopping=True,
    max_concurrent_iterations=4)

In [12]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on Saturn-AI with default configuration
Running on remote compute: Saturn-AI


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-azure-capstone-project,AutoML_665cb8fb-2826-4b41-a971-2e6ac8178cfc,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-azure-capstone-project,AutoML_665cb8fb-2826-4b41-a971-2e6ac8178cfc,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_665cb8fb-2826-4b41-a971-2e6ac8178cfc',
 'target': 'Saturn-AI',
 'status': 'Completed',
 'startTimeUtc': '2021-04-19T11:09:45.521741Z',
 'endTimeUtc': '2021-04-19T11:39:14.406235Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'Saturn-AI',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-azure-capstone-project","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group":"aml-quickstarts-143143","workspace_name":"quick-starts-ws-143143","region":"southcentralus","compute_target":"Saturn-AI","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"validatio

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [33]:
best_run, model = remote_run.get_output()
best_run_tags = best_run.get_tags()
best_run_metrics = best_run.get_metrics()

WARNING - The model you attempted to retrieve requires 'azureml-train-automl-runtime' to be installed at '==1.26.0'. Please install 'azureml-train-automl-runtime==1.26.0' (e.g. `pip install azureml-train-automl-runtime==1.26.0`) and then rerun the previous command.


In [34]:
best_run_tags

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":2,"CurrentNodeCount":4}',
 'mlflow.source.type': 'JOB',
 'mlflow.source.name': 'automl_driver.py',
 'model_explain_run_id': 'AutoML_665cb8fb-2826-4b41-a971-2e6ac8178cfc_ModelExplain',
 'model_explanation': 'True'}

In [35]:
best_run_metrics

{'AUC_micro': 0.9184999999999999,
 'precision_score_macro': 0.8920874584874939,
 'average_precision_score_micro': 0.9187092096434253,
 'f1_score_macro': 0.8736199353413496,
 'average_precision_score_weighted': 0.9192310678704205,
 'log_loss': 0.3772695009957162,
 'balanced_accuracy': 0.8672161172161171,
 'matthews_correlation': 0.7578833083374518,
 'accuracy': 0.89,
 'AUC_macro': 0.9082264957264957,
 'AUC_weighted': 0.9082264957264957,
 'f1_score_weighted': 0.8891644340994013,
 'norm_macro_recall': 0.7344322344322344,
 'recall_score_weighted': 0.89,
 'f1_score_micro': 0.89,
 'recall_score_macro': 0.8672161172161171,
 'average_precision_score_macro': 0.9020510767382299,
 'recall_score_micro': 0.89,
 'weighted_accuracy': 0.9077487037523717,
 'precision_score_weighted': 0.8979478414615991,
 'precision_score_micro': 0.89,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_665cb8fb-2826-4b41-a971-2e6ac8178cfc_53/accuracy_table',
 'confusion_matrix': 'aml://artifactId/ExperimentR

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [46]:
model = best_run.register_model(model_path='outputs/model.pkl', model_name='automl',
                        tags={'Name':'AutoML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

In [47]:
# best_run.download_file(constants.CONDA_ENV_FILE_PATH, "env.yml")
# environment = best_run.get_context().get_environment()

best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py')

In [48]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-143143', subscription_id='9a7511b8-150f-4a58-8528-3e7d50216c31', resource_group='aml-quickstarts-143143'), name=automl, id=automl:3, version=3, tags={'Name': 'AutoML'}, properties={'AUC': '0.9082264957264957', 'Accuracy': '0.89'})

In [49]:
# env = Environment.from_conda_specification(name="env",file_path="./env.yml")

inference_config = InferenceConfig(entry_script="scoring.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1)

aci_service_name = 'automl-service'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-19 12:39:02+00:00 Creating Container Registry if not exists..
2021-04-19 12:39:14+00:00 Registering the environment.
2021-04-19 12:39:15+00:00 Building image..
2021-04-19 12:51:22+00:00 Generating deployment configuration.
2021-04-19 12:51:24+00:00 Submitting deployment to compute..
2021-04-19 12:51:30+00:00 Checking the status of deployment automl-service..
2021-04-19 12:57:50+00:00 Checking the status of inference endpoint automl-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [52]:
service = aci_service

In [53]:
scoring_uri = service.scoring_uri

In [54]:
print(f'Service status: {service.state}')
print(f'Scoring URI: {service.scoring_uri}')
print(f'swagger URI: {service.swagger_uri}')

Service status: Healthy
Scoring URI: http://3365387e-88aa-42c1-92c2-82bf4da73f02.southcentralus.azurecontainer.io/score
swagger URI: http://3365387e-88aa-42c1-92c2-82bf4da73f02.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [55]:
import requests
import json

payload ={"data":
  [{'age': 36.0,
  'anaemia': 0,
  'creatinine_phosphokinase': 981,
  'diabetes': 1,
  'ejection_fraction': 55,
  'high_blood_pressure': 1,
  'platelets': 368000,
  'serum_creatinine': 3.5,
  'serum_sodium': 137,
  'sex': 1,
  'smoking': 0,
  'time': 4}]
  }

input_data = json.dumps(payload)
with open("data.json", "w") as _f:
    _f.write(input_data)
headers = {'Content-Type': 'application/json'}
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": [1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [57]:
print(service.get_logs())

2021-04-19T12:57:39,715110100+00:00 - gunicorn/run 
2021-04-19T12:57:39,727638200+00:00 - iot-server/run 
2021-04-19T12:57:39,727981400+00:00 - rsyslog/run 
2021-04-19T12:57:39,760052000+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libuuid.so.1: no version information available (required by rsyslogd)
/usr/sbin/nginx: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [ ]:
# inference_config = InferenceConfig(entry_script="scoring.py", environment=myenv)
# aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
# service = Model.deploy(workspace=ws,
#                       name="automl-service",
#                       models=[model],
#                       inference_config=inference_config,
#                       deployment_config=aci_config,
#                       overwrite=True)
# service.wait_for_deployment(show_output=True)